# Loop Inference with Camera and Threads

Loop is run with 4 threads and images are plotted inside asyncio loop:

1. ```get_frames```:
    - Retrieve images from the camera at the desired fps.
    - It warms up the camera taking a few images first, controlled by ```WARM_UP_FRAMES```.
    - It blocks until warm up frames are plotted.
2. ```do_batches```: 
    - Takes images from get_frames and create batches for the accelerator.
    - It also sends images to plot.
3. ```accel_exec```: 
    - Asynchronous execution of the accelerator.
4. ```check_dma_ready```: 
    - Polls the DMA to check if accelerator has finished the batch prediction. 
    
- ```plot_frames```:
    - asyncio function to plot frames.
    - It plots all warm up frames.
    - It plots a portion of images inside the batch.
    - It computes the mean of the batch prediction, an uses it as a final prediction for the whole batch.

In [1]:
import os
import numpy as np

from threading import Thread, Event
from queue import Queue
import asyncio

import cv2
import opencv_jupyter_ui as jcv2

from datetime import datetime, timedelta
import time

import driver
from pynq.pl_server.device import Device

# Config

In [2]:
FRAME_IN_W = 640        # Image width to capture by camera
FRAME_IN_H = 480        # Image height to capture by camera
N_FRAMES = 8000         # Number of images to predict

IMG_W_RSZ = 224         # Image width resize for the accelerator
IMG_H_RSZ = 224         # Image height resize for the accelerator
BATCH_SIZE = 20         # Be careful if Camera does not provide xxx FPS, as time computation may be wrong
FPGA_CLK = 4.0          # FPGA clock

CAP_FPS = BATCH_SIZE    # fps to capture

WARM_UP_FRAMES = 5
NUM_FRAMES_TO_PLOT = 3  # Plot 2 images of every batch

VERBOSE = False
# Sleep if there are no new frames from the camera
SLEEP_BATCH_FRAME = 0.01
# Sleep DMA while inference is calculated: used for polling DMA
SLEEP_DMA_POLL = 0.2

# Accelerator

There are two new functions in the accelerator, to run asynchronously and poll the DMA for the result. This could be done with interruptions too, adding them to Vivado Block Design.

```python
def async_exec(self, input_npy)
def polling_out_dma_ready(self)
```

In [3]:
my_device = Device.devices[0]

accel = driver.FINNExampleOverlay(
    bitfile_name = '../bitfile/finn-accel.bit', 
    platform = "zynq-iodma",
    io_shape_dict = driver.io_shape_dict, 
    batch_size = BATCH_SIZE,
    fclk_mhz = FPGA_CLK,
    runtime_weight_dir = "runtime_weights/", 
    device=my_device
)

Asynch = False -> block until finished


### Initial Throughput Test

In [4]:
res = accel.throughput_test()
file = open("nw_metrics_threads.txt", "w")
file.write(str(res))
file.close()
print("Results written to nw_metrics.txt")

Asynch = False -> block until finished
Results written to nw_metrics.txt


# Camera Setup

In [5]:
def init_videocapture(width=1280, height=720, fps = 30):
    camera = cv2.VideoCapture(0, cv2.CAP_V4L2)
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
    camera.set(cv2.CAP_PROP_FPS, fps)
    return camera

# Camera Thread

In [6]:
def get_frames(
    event_warm_up_ready,
    event_initial_plot_ready,
    queue_warm_up,
    queue_frames,
    frame_in_w, 
    frame_in_h, 
    n_frames,
    batch_size,
    warm_up_frames = 5,
    cap_fps = 30,
    verbose = False):
    
    '''
    Get frames continuously from the camera.
    '''
    
    capture = init_videocapture(width=frame_in_w, height=frame_in_h, fps=cap_fps)
    print(f'$ Camera Thread: Is webcam open: {capture.isOpened()}')
    img_w = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    img_h = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    cv_fps = capture.get(cv2.CAP_PROP_FPS) 
    print(f'$ Camera Thread: Camera (width, height) = ({img_w}, {img_h}) - FPS = {cv_fps}')
    
    ################################
    #           Warm Up            #
    ################################
    warm_up_count = 0
    while (warm_up_count < warm_up_frames):
        ret, frame = capture.read()
        queue_warm_up.put(frame)
        warm_up_count += 1
    event_warm_up_ready.set()
    print(f'\n$ Camera Thread: Warm up of {warm_up_frames} frames finished\n')
    print(f'\n$ Camera Thread: Waiting for Initial Plot\n')
    event_initial_plot_ready.wait()
    print(f'\n$ Camera Thread: Initial Plot finished\n')
    
    ################################
    #            Loop              #
    ################################
    img_count = 0
    start = datetime.now()
    while (img_count < n_frames):
        ret, frame = capture.read()
        queue_frames.put(frame)
        if verbose == True:
            img_in_batch = img_count % batch_size
            print(f'$ Camera Thread: got image = {img_count} - Id in batch: {img_in_batch} - Queue frames size = {queue_frames.qsize()}.')
        img_count += 1
    end = datetime.now()
    
    queue_frames.put(None) # Signal plot_frames to stop
    
    delta_secs = (end-start).total_seconds()
    fps = round(n_frames / delta_secs, 1)
    print("\n$ *******************************")
    print(f'$ Camera Thread: FPS = {fps:.1f}')
    print("$ *******************************\n")
    
    capture.release()

# Batches Thread

Fills the Queue to plot ```NUM_FRAMES_TO_PLOT``` images per inference. It computes the interval based in ```BATCH_SIZE```.

Example:
- Batch of 20 images.
- ```NUM_FRAMES_TO_PLOT = 3```

$$
Interval = \lceil \frac{Batch}{Plot_{Frames}} \rceil = \lceil \frac{20}{3} \rceil = 7
$$

Therefore, it will plot image index 0, 7 and 14.

In [8]:
def do_batches(
    queue_frames, 
    queue_batches, 
    event_batch_ready, 
    batch_size,
    queue_frames_2_plot,
    num_frames_2_plot,
    sleep_time = 0.02,
    verbose = False): #, queue_frames_2_plot):
    
    '''
    Do batches taking images from Queue Frames. Sleep if there are no frames in the queue.
        - BGR2RGB
        - Resize
        - Expand dims to add batch dim
    Once a batch is formed, put it in Queue Batches.
    Queue Batches is of size 1, to block if previous batch was not predicted by the accelerator.
    
    Put num_frames_2_plot images of the batch in the queue_frames_2_plot, to plot it afterwards.
    '''
    
    batch_idx = 0
    img_id = 0
    assert num_frames_2_plot <= batch_size, f'Please, {num_frames_2_plot} <= {batch_size}'
    img_2_plot_interval = int(batch_size / num_frames_2_plot) + 1
    frames_2_plot_list = []
    
    while True:
        if queue_frames.empty():
            if verbose == True:
                print(">> Batches Thread: empty queue frames from camera")
            time.sleep(sleep_time)
        else:
            frame = queue_frames.get()
            if verbose == True:
                print(f">> Batches Thread: image {img_id} for batch {batch_idx} - Queue batches size = {queue_batches.qsize()}.")
            if frame is None:
                break 
            # Form batches for accel prediction
            img = frame.copy()
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_W_RSZ, IMG_H_RSZ), interpolation = cv2.INTER_LINEAR)
            assert img.dtype == "uint8", "Image datatype must be UINT8"
            img = np.expand_dims(img, axis=0)
            if img_id == 0:
                # To plot images inside notebook
                frames_2_plot_list.append(frame)
                if verbose == True:
                    print(f">> Batches Thread: plot frame added to batch {batch_idx} - Queue plot size = {queue_frames_2_plot.qsize()}.")
                batch_imgs = img
            else:
                batch_imgs = np.concatenate((batch_imgs, img), axis=0)
                if img_id % img_2_plot_interval == 0:
                    frames_2_plot_list.append(frame)
            if img_id == (batch_size - 1):
                # Put blocks if accel did not get previous batch
                queue_batches.put([batch_idx, batch_imgs])
                # Put list of frames to plot
                print(f">> Batches Thread: sending {len(frames_2_plot_list)} to plot.")
                queue_frames_2_plot.put([batch_idx, frames_2_plot_list])
                if verbose == True:
                    print(f">> Batches Thread: Batch {batch_idx} for accel - Queue batches size = {queue_batches.qsize()}.")
                event_batch_ready.set()
                batch_idx += 1
                # Empty list of frames 2 plot, to use with next batch
                frames_2_plot_list = []
            img_id = (img_id + 1) % batch_size
            queue_frames.task_done()
    
    # Signal queues to stop
    print(">> Batches Thread: No more batches to do. Put None.")
    queue_batches.put([batch_idx, None])
    queue_frames_2_plot.put([batch_idx, None])
    time.sleep(5)
    event_batch_ready.set()

# Accel Thread

In [9]:
def accel_exec(
    queue_batches, 
    event_batch_ready, 
    event_accel_exec, 
    event_yhat_ready_accel,
    verbose = False):
    
    '''
    Asynchronous execution of the accelerator.
    It waits for a batch ready.
    It blocks until previous prediction is retrieved by the DMA.
    '''
    
    batch_idx = 0
    
    while True:
        event_batch_ready.wait()
        event_batch_ready.clear()
        # It avoids getting the new batch if previous one was not retrieved from DMA first
        # Therefore, this thread blocks until previous inference is performed
        # It causes that do_batches and get_frames gets blocked too
        event_yhat_ready_accel.wait()
        event_yhat_ready_accel.clear()
        do_batch_idx, batch = queue_batches.get()
        assert do_batch_idx == batch_idx, f"@ Accel Thread: do batch idx {do_batch_idx} does not match accel batch idx {batch_idx}."
        if batch is None:
            print("@@@ Accel Thread: No more Batches signal received.")
            break
        if verbose == True:
            print(f'@@@ Accel Thread: Batch idx {batch_idx}. Batch shape = {batch.shape}.')
        # Execute accelerator
        if verbose == True:
            start_accel = datetime.now()
        accel.async_exec([batch])
        # Notify DMA Poll that it can start polling for prediction ready
        if verbose == True:
            end_accel = datetime.now()
            print(f'@@@ Accel Thread: time to exec accel {(end_accel-start_accel).microseconds/1e3:.2f} [ms] - {end_accel}')   
        event_accel_exec.set()
        batch_idx += 1  
        queue_batches.task_done()
        
    queue_batches.task_done()

# Check DMA Result Thread

In [10]:
def check_dma_ready(
    event_accel_exec, 
    queue_yhat, 
    event_yhat_ready,
    event_yhat_ready_accel,
    n_frames, 
    batch_size,
    sleep_time = 0.01,
    verbose = False):
    
    '''
    Polls the DMA to check if prediction was already completed by the accelerator.
    It signals the accelerator that prediction was completed, so it can load next batch: event_yhat_ready_accel.
    It signals the main thread that prediction is ready, so it can plot it: event_yhat_ready. 
    
    With this strategy, it blocks do_batches, as the accelerator cannot accept new batches until previous one is completed.
    '''
    
    total_batches = int(n_frames / batch_size)
    batch_idx = 0
    
    accel_preds_times = []
    
    while (batch_idx < total_batches):
        event_accel_exec.wait()
        start_accel = datetime.now()           
        event_accel_exec.clear()
        if verbose == True:
            print(f"**** DMA Thread: Accel started Execution - {start_accel}.")
        while True:
            if verbose == True:
                now_time = datetime.now()   
                print(f"**** DMA Thread: ... Polling for DMA result ... : {now_time}")
            out_ready, yhat = accel.polling_out_dma_ready()
            if out_ready == True:
                queue_yhat.put([batch_idx, yhat])
                
                if verbose == True:
                    print("\nOutput DMA ready")
                    print(f'{yhat}')
                    # Calculate time between predictions
                    end_accel = datetime.now()
                    if batch_idx == 0:
                        accel_preds_times.append(end_accel)
                    else:
                        accel_preds_times.append(end_accel)
                        time_between_preds = accel_preds_times[1] - accel_preds_times[0]
                        if batch_idx == 1:
                            accel_preds_sum = time_between_preds
                        else:
                            accel_preds_sum = accel_preds_sum + time_between_preds
                        accel_preds_mean_time = accel_preds_sum / batch_idx
                        accel_preds_times.pop(0)
                        print(f'\n_____ Accel between predictions: {time_between_preds.total_seconds():.2f} [secs] _____')
                        print(f'_____ Mean Accel between predictions: {accel_preds_mean_time.total_seconds():.2f} [secs] _____')
                    print(f'_____ Accel elapsed time: {(end_accel-start_accel).microseconds/1e3:.2f} [ms] _____\n')   
                
                # Print time between predictions and accel elapsed time close to plot window
                else:
                    end_accel = datetime.now()
                    if batch_idx == 0:
                        accel_preds_times.append(end_accel)
                        str_2_print = ""
                    else:
                        accel_preds_times.append(end_accel)
                        time_between_preds = accel_preds_times[1] - accel_preds_times[0]
                        if batch_idx == 1:
                            accel_preds_sum = time_between_preds
                        else:
                            accel_preds_sum = accel_preds_sum + time_between_preds
                        accel_preds_mean_time = accel_preds_sum / batch_idx
                        accel_preds_times.pop(0)
                        str_2_print = f'\nBatch idx = {batch_idx}\n'
                        str_2_print += f'--- Accel between predictions: {time_between_preds.total_seconds():.2f} [secs]\n'
                        str_2_print += f'*** Mean Accel between predictions: {accel_preds_mean_time.total_seconds():.2f} [secs]\n'
                    str_2_print += f'... Accel elapsed time: {(end_accel-start_accel).microseconds/1e3:.2f} [ms]\n'
                    print(str_2_print, end='\r')
                
                event_yhat_ready.set()
                # Finish blocking of accel if needed
                event_yhat_ready_accel.set()
                batch_idx += 1
                break
            else:
                if verbose == True:
                    print(f"**** DMA Thread: yhat not ready. Sleep {sleep_time} [secs].")
                time.sleep(sleep_time)
            
    # Signal queues to stop
    print("**** DMA Thread: No more batches to do. Put None.")
    queue_yhat.put([batch_idx, None])
    time.sleep(5)
    event_yhat_ready.set() 

# Plot Thread

#### Colors

In [11]:
GRAY_COLOR = (50, 50, 50)      # No Smoke, No Fire
RED_COLOR = (0,0,255)          # Only Fire
BLUE_COLOR = (255,255,0)       # Only Smoke
YELLOW_COLOR = (0,255,255)     # Smoke & Fire

#### Draw Prediction in Frame Function

In [12]:
def draw_pred_box(yhat, img_to_plot, img_w, img_h):

    yhat_str = np.array2string(yhat)
    empty_str = "Empty: " + yhat_str
    smoke_str = "Smoke: " + yhat_str
    fire_str = "Fire: " + yhat_str
    smoke_fire_str = "Smoke & Fire: " + yhat_str

    # Empty
    if yhat[0] < 0.5 and yhat[1] < 0.5:
        cv2.rectangle(img_to_plot, (0,0), (img_w, img_h), GRAY_COLOR, 20)
        cv2.rectangle(img_to_plot, (0,0), (img_w, 35), GRAY_COLOR, -1)
        cv2.putText(img_to_plot, empty_str, (8, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, WHITE_COLOR, 1) 
    # Smoke
    elif yhat[0] > 0.5 and yhat[1] < 0.5:
        cv2.rectangle(img_to_plot, (0,0), (img_w, img_h), BLUE_COLOR, 20)
        cv2.rectangle(img_to_plot, (0,0), (img_w, 35), BLUE_COLOR, -1)
        cv2.putText(img_to_plot, smoke_str, (8, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, BLACK_COLOR, 1) 
    # Fire
    elif yhat[0] < 0.5 and yhat[1] > 0.5:
        cv2.rectangle(img_to_plot, (0,0), (img_w, img_h), RED_COLOR, 20)
        cv2.rectangle(img_to_plot, (0,0), (img_w, 35), RED_COLOR, -1)
        cv2.putText(img_to_plot, fire_str, (8, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, BLACK_COLOR, 1) 
    # Smoke & Fire
    elif yhat[0] > 0.5 and yhat[1] > 0.5:
        cv2.rectangle(img_to_plot, (0,0), (img_w, img_h), YELLOW_COLOR, 20)
        cv2.rectangle(img_to_plot, (0,0), (img_w, 35), YELLOW_COLOR, -1)
        cv2.putText(img_to_plot, smoke_fire_str, (8, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, BLACK_COLOR, 1)
        
    return img_to_plot

#### Plot several frames per inference

The number of frames can be defined in do_batches

In [14]:
async def plot_frames(
    event_warm_up_ready,
    event_initial_plot_ready,
    queue_warm_up,
    event_yhat_ready,
    queue_yhat,
    queue_frames_2_plot):
    
    '''
    It waits the warm up signal and plot 1 frame.
    Once a prediction is ready, it computes the mean and plot the images with color boxes.
    '''

    ################################
    #           Warm Up            #
    ################################
    event_warm_up_ready.wait()
    print(':):):) Plot Thread: start Initial Plot.')    
    for i in range(queue_warm_up.qsize()):
        new_frame = queue_warm_up.get()
        if i == 0:
            jcv2.imshow('UAV', new_frame)
        queue_warm_up.task_done()
    event_initial_plot_ready.set()
    
    ################################
    #            Loop              #
    ################################
    plot_batch_idx = 0

    while True:
        event_yhat_ready.wait()
        event_yhat_ready.clear()
        yhat_batch_idx, yhat = queue_yhat.get()
        plot_frames_idx, frames_to_plot_list = queue_frames_2_plot.get()
        assert yhat_batch_idx == plot_batch_idx, f':) Plot Thread: plot batch idx {plot_batch_idx} does not match yhat batch idx {yhat_batch_idx}.'
        assert plot_frames_idx == plot_batch_idx, f':) Plot Thread: plot frames idx {plot_frames_idx} does not match batch idx {yhat_batch_idx}.'
        if yhat is None:
            print(':):):) Plot Thread: yhat is None received -> Execution Finished')
            break
        else:
            yhat_mean = np.mean(yhat, axis=0)
            smoke_mean = yhat_mean[0]
            fire_mean = yhat_mean[1]
            if VERBOSE:
                print(f':):):) Plot Thread: yhat\n {yhat}')
                print(f':):):) Plot Thread: smoke mean = {smoke_mean}')
                print(f':):):) Plot Thread: fire mean = {fire_mean}')
                print(f':):):) Plot Thread: frames to plot list length = {len(frames_to_plot_list)} \n')
            for frame in frames_to_plot_list:
                frame_to_plot = draw_pred_box(yhat_mean, frame, FRAME_IN_W, FRAME_IN_H)   
                jcv2.imshow('UAV', frame_to_plot)
                if jcv2.waitKey(1)==ord('q'):
                    break 
                time.sleep(0.3)
            plot_batch_idx += 1
            queue_yhat.task_done()
            queue_frames_2_plot.task_done()

    queue_yhat.task_done()
    queue_frames_2_plot.task_done()
    jcv2.destroyAllWindows()

# Threads Communication: Queues and Events

### Queues
- ```queue_warm_up```:
    - get_frames fills it for plot_frames.
- ```queue_frames```: 
    - get_frames fills it and do_batches consumes it.
- ```queue_batches```
    - do_batches  fills it and accel_exec consumes it.
    - size = 1, so it blocks do_batches if accel_exec did not predict the previous batch yet.
- ```queue_yhat```:
    - check_dma_ready fills it and plot_frames consumes it.
    - size = 1, so it blocks check_dma_ready if accel_exec did not predict the previous batch yet.
- ```queue_frames_2_plot```:
    - do_batches fills it and plot_frames consumes it.
    
### Events
- ```event_warm_up_ready```:
    - get_frames notifies that warm up is finished, so plot_frames can plot the images.
- ```event_initial_plot_ready```:
    - plot_frames notifies that warm up is finished, so get_frames can retrieve images for the accelerator.
- ```event_batch_ready```:
    - do_batches notifies that new batch is ready for the accelerator.
- ```event_accel_exec```:
    - accel_exec notifies that it is starting the prediction of a new batch.
    - It triggers check_dma_ready, so it starts polling for the prediction.
- ```event_yhat_ready_accel```:
    - check_dma_ready notifies that prediction is ready, so the accelerator is free for new batches.
    - It is initialized = 1, as the accelerator starts available.
- ```event_yhat_ready```:
    - check_dma_ready notifies that prediction is ready, so plot_frames can plot the prediction.

### Setup Queues

In [15]:
queue_warm_up = Queue(maxsize=WARM_UP_FRAMES)

queue_frames = Queue(maxsize=BATCH_SIZE*2) # -> BATCH_SIZE*2 is very conservative, it should be reduced
queue_batches = Queue(maxsize=1)
queue_yhat = Queue(maxsize=1)
queue_frames_2_plot = Queue(maxsize=8)

### Setup Events

In [16]:
event_warm_up_ready = Event()
event_initial_plot_ready = Event()

event_batch_ready = Event()
event_accel_exec = Event()
event_yhat_ready_accel = Event()
event_yhat_ready_accel.set() # To let first batch start in the accelerator
event_yhat_ready = Event()

### Setup Threads

In [17]:
cap_thread = Thread(
    target=get_frames, 
    args=(
        event_warm_up_ready, event_initial_plot_ready, queue_warm_up,
        queue_frames, FRAME_IN_W, FRAME_IN_H, N_FRAMES, BATCH_SIZE, WARM_UP_FRAMES, CAP_FPS, VERBOSE,))

batches_thread = Thread(
    target=do_batches, 
    args=(queue_frames, queue_batches, event_batch_ready, BATCH_SIZE, 
          queue_frames_2_plot, 
          NUM_FRAMES_TO_PLOT,
          SLEEP_BATCH_FRAME, VERBOSE,))

accel_thread = Thread(
    target=accel_exec, 
    args=(queue_batches, event_batch_ready, event_accel_exec, event_yhat_ready_accel, VERBOSE,))

check_dma_ready = Thread(
    target=check_dma_ready, 
    args=(event_accel_exec, queue_yhat, event_yhat_ready, event_yhat_ready_accel, N_FRAMES, BATCH_SIZE, SLEEP_DMA_POLL, VERBOSE,))

### Asyncio Thread to Plot

In [18]:
def run_event_loop(loop):
    # report a message
    print('Asyncio event loop is running')
    # set the loop for the current thread
    asyncio.set_event_loop(loop)
    # run the event loop until stopped
    loop.run_forever()
    
# create a new event loop (low-level api)
loop = asyncio.new_event_loop()

plot_thread = Thread(target=run_event_loop, args=(loop,), daemon=True)

### Start Threads

In [19]:
start_running = datetime.now()

In [20]:
cap_thread.start()
batches_thread.start()
accel_thread.start()
check_dma_ready.start()

In [21]:
plot_thread.start()
print("\nWaiting for capture thread")

future = asyncio.run_coroutine_threadsafe(plot_frames(
                                                event_warm_up_ready, event_initial_plot_ready, queue_warm_up,
                                                event_yhat_ready,
                                                queue_yhat,
                                                queue_frames_2_plot), 
                                            loop)
# wait for the task to finish
value = future.result()
# report a message
print(f'Got Async Result: {value}')

Asyncio event loop is running
Waiting for capture thread

$ Camera Thread: Is webcam open: True
$ Camera Thread: Camera (width, height) = (640.0, 480.0) - FPS = 20.0

$ Camera Thread: Warm up of 5 frames finished
:):):) Plot Thread: start Initial Plot.


$ Camera Thread: Waiting for Initial Plot




$ Camera Thread: Initial Plot finished

... Accel elapsed time: 848.84 [ms]

Batch idx = 1
--- Accel between predictions: 1.17 [secs]
*** Mean Accel between predictions: 1.17 [secs]
... Accel elapsed time: 846.62 [ms]

Batch idx = 2
--- Accel between predictions: 0.93 [secs]
*** Mean Accel between predictions: 1.05 [secs]
... Accel elapsed time: 829.82 [ms]

Batch idx = 3
--- Accel between predictions: 0.98 [secs]
*** Mean Accel between predictions: 1.03 [secs]
... Accel elapsed time: 835.63 [ms]

Batch idx = 4
--- Accel between predictions: 0.99 [secs]
*** Mean Accel between predictions: 1.02 [secs]
... Accel elapsed time: 834.53 [ms]

Batch idx = 5
--- Accel between predictions: 1.00 [secs]
*** Mean Accel between predictions: 1.01 [secs]
... Accel elapsed time: 830.18 [ms]

Batch idx = 6
--- Accel between predictions: 1.04 [secs]
*** Mean Accel between predictions: 1.02 [secs]
... Accel elapsed time: 828.57 [ms]

Batch idx = 7
--- Accel between predictions: 1.02 [secs]
*** Mean Acce


KeyboardInterrupt




Batch idx = 349
--- Accel between predictions: 1.34 [secs]
*** Mean Accel between predictions: 1.01 [secs]
... Accel elapsed time: 708.41 [ms]


### Wait for Threads

In [ ]:
# cap_thread.join()
# batches_thread.join()
# accel_thread.join()

# Main Thread Loop

### Plot inside Asyncio Loop Thread

In [ ]:
# plot_thread.join()
    
end_running = datetime.now()

print(f'\nElapsed time = {end_running - start_running}.')

### Plot in Main Thread

In [ ]:
# '''
# When yhat is ready, it retrives a frame from Queue Frames to Plot and yhat from Queue Yhat.
# Calculates the mean of yhat for both classes and plot it.
# '''

# plot_batch_idx = 0

# while True:
#     event_yhat_ready.wait()
#     event_yhat_ready.clear()
#     yhat_batch_idx, yhat = queue_yhat.get()
#     plot_frames_idx, frame_to_plot = queue_frames_2_plot.get()
#     assert yhat_batch_idx == plot_batch_idx, f':) Main Thread: plot batch idx {plot_batch_idx} does not match yhat batch idx {yhat_batch_idx}.'
#     assert plot_frames_idx == plot_batch_idx, f':) Main Thread: plot frames idx {plot_frames_idx} does not match batch idx {yhat_batch_idx}.'
#     if yhat is None:
#         print(':):):) Main Thread: yhat is None received -> Execution Finished')
#         break
#     else:
#         smoke_mean = np.mean(yhat[:, 0])
#         fire_mean = np.mean(yhat[:, 1])
#         if VERBOSE:
#             print(f':):):) Main Thread: yhat\n {yhat}')
#             print(f':):):) Main Thread: smoke mean = {smoke_mean}')
#             print(f':):):) Main Thread: fire mean = {fire_mean}')
#             # print(f':):):) Main Thread: yhat\n {type(yhat)}')
#             # print(f':):):) Main Thread: yhat\n {yhat.shape}')
#         jcv2.imshow('UAV', frame_to_plot)
#         if jcv2.waitKey(1)==ord('q'):
#             break 
#         plot_batch_idx += 1
#         queue_yhat.task_done()
#         queue_frames_2_plot.task_done()
        
# queue_yhat.task_done()
# queue_frames_2_plot.task_done()
# jcv2.destroyAllWindows()

# end_running = datetime.now()

# print(f'\nElapsed time = {end_running - start_running}.')